<a href="https://colab.research.google.com/github/atick-faisal/GRF-Synthesis-from-Motion-Trajectories/blob/main/src/training/gft_synthesis_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !gdown --id "1kNGwWBQp6kmAioLP-_zWx7r42SzoEO3v"
# !unzip -u UNet.zip

# DB2
# !gdown --id "19ysOVgso3enZZTCeCmKTouBdRo1UFSz0"

In [3]:
# !pip install --upgrade pandas==1.3.4

In [4]:
import h5py
import scipy
import random
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops

from UNet_1DCNN import UNet
from FPN_1DCNN import FPN
from AlbuNet_1DCNN import AlbUNet

%matplotlib inline
sns.set(font_scale=1.5)
sns.set_style("darkgrid", {'font.family': 'serif',
              'font.serif': 'Times New Roman'})

models_dir = '/content/drive/MyDrive/Research/GRF Data Synthesis/Results/DB2/Force/Models/X/'
figures_dir = '/content/drive/MyDrive/Research/GRF Data Synthesis/Results/DB2/Force/Figures/X/'


In [5]:
def prepareTrainDict(y, model_depth, signal_length, model_name):
    def approximate(inp, w_len, signal_length):
        op = np.zeros((len(inp), signal_length//w_len))
        for i in range(0, signal_length, w_len):
            try:
                op[:, i//w_len] = np.mean(inp[:, i:i+w_len], axis=1)
            except Exception as e:
                print(e)
                print(i)

        return op

    out = {}
    Y_Train_dict = {}
    out['out'] = np.array(y)
    Y_Train_dict['out'] = out['out']
    for i in range(1, (model_depth+1)):
        name = f'level{i}'
        if ((model_name == 'UNet') or (model_name == 'MultiResUNet') or (model_name == 'FPN')):
            out[name] = np.expand_dims(approximate(
                np.squeeze(y), 2**i, signal_length), axis=2)
        elif ((model_name == 'UNetE') or (model_name == 'UNetP') or (model_name == 'UNetPP')):
            out[name] = np.expand_dims(approximate(
                np.squeeze(y), 2**0, signal_length), axis=2)
        Y_Train_dict[f'level{i}'] = out[f'level{i}']

    return out, Y_Train_dict


def Construction_Error(GRND, Pred):
    construction_err = []
    rmse = []
    corr_coef = []
    corr_coef2 = []
    bad_indices = []
    count = 0

    for i in range(len(GRND)):
        MAE = np.mean(np.abs(Pred[i].ravel() - GRND[i].ravel()))
        RMSE = mean_squared_error(np.nan_to_num(
            Pred[i].ravel()), GRND[i].ravel(), squared=False)
        R = np.corrcoef(np.nan_to_num(Pred[i].ravel()), GRND[i].ravel())[0, 1]
        R2 = pearsonr(np.nan_to_num(Pred[i].ravel()), GRND[i].ravel())[0]
        if MAE < 1:
            construction_err.append(MAE)
            rmse.append(RMSE)
            corr_coef.append(R)
            corr_coef2.append(R2)

        elif MAE >= 1:
            count = count + 1
            bad_indices.append(i)

    print(
        f'Construction Error : {round(np.mean(construction_err), 3)} +/- {round(np.std(construction_err), 3)}')
    print(f'RMSE : {round(np.mean(rmse), 3)} +/- {round(np.std(rmse), 3)}')
    print(
        f'R1 : {round(np.mean(corr_coef), 3)} +/- {round(np.std(corr_coef), 3)}')
    print(
        f'R2 : {round(np.mean(corr_coef2), 3)} +/- {round(np.std(corr_coef2), 3)}')
    print(f'Number of Bad Predictions = {count}')

    bad_indices = set(bad_indices)
    all_indices = set(np.arange(len(GRND)))
    good_indices = np.array(list(all_indices - bad_indices))
    GRND_NEW = GRND[good_indices]
    PRED_NEW = Pred[good_indices]

    return construction_err, rmse, corr_coef, corr_coef2

def history_plot(history):
        # list all dictionaries in history
        print(history.history.keys())
        # summarize history for error
        plt.figure(figsize=(11,14))
        plt.subplot(2,1,1)
        plt.plot(history.history['out_mean_squared_error'], linewidth=2)
        plt.plot(history.history['val_out_mean_squared_error'], linewidth=2)
        plt.title('Model Error Performance')
        plt.ylabel('Error (MSE)')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Val'], loc='upper right')
        #   plt.ylim([0, 3])
        # plt.savefig(figures_dir + 'EC_' + config + '.png')
        # plt.show()
        # summarize history for loss
        # plt.figure(figsize=(11,7))
        plt.subplot(2,1,2)
        plt.plot(history.history['out_loss'], linewidth=2)
        plt.plot(history.history['val_out_loss'], linewidth=2)
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Val'], loc='upper right')
        #   plt.ylim([0, 3])
        plt.savefig(figures_dir + 'LC_' + config + '.png')
        plt.show()


In [6]:
model_name = 'MultiResUNet'  # UNet or UNetPP
# signal_length = FRAME_LEN  # Length of each Segment
model_depth = 5  # Number of Level in the CNN Model
model_width = 64  # Width of the Initial Layer, subsequent layers start from here
kernel_size = 3  # Size of the Kernels/Filter
# num_channel = N_CHANNELS  # Number of Channels in the Model
D_S = 1  # Turn on Deep Supervision
A_E = 0  # Turn on AutoEncoder Mode for Feature Extraction
A_G = 0  # Attention Guided
problem_type = 'Regression'
# Number of Class for Classification Problems, always '1' for Regression Problems
output_nums = 1
'''Only required if the AutoEncoder Mode is turned on'''
feature_number = 1024  # Number of Features to be Extracted
'''Only required for MultiResUNet'''
alpha = 1  # Model Width Expansion Parameter

EPOCHS = 300


# 5 Fold CV

In [7]:
data = joblib.load("/content/data2_f15_t6_n1_all_vel.joblib")
features = data['X']
target = data['y']

X = features.to_numpy().reshape(-1, 1024, 15)
y = target.to_numpy().reshape(-1, 1024, 6)

print(X.shape)
print(y.shape)

X = X[:, :, [10, 11, 12, 13, 14]]
y = y[:, :, 0] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

X = np.nan_to_num(X)
y = np.nan_to_num(y)

(3920, 1024, 15)
(3920, 1024, 6)


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=700)
fold = 1

results = pd.DataFrame()

for train_idx, test_idx in kf.split(X):
    X_Train, X_Test = X[train_idx], X[test_idx]
    Y_Train, Y_Test = y[train_idx], y[test_idx]

    print(f"==================================== FOLD {fold} ====================================")

    print(X_Train.shape)
    print(X_Test.shape)
    print(Y_Train.shape)
    print(Y_Test.shape)

    FRAME_LEN = X_Train.shape[1]
    N_CHANNELS = X_Train.shape[2]
    signal_length = FRAME_LEN
    num_channel = N_CHANNELS

    #########################################################################################
    X_Train1 = X_Train
    X_Test1 = X_Test
    [Y_Train1, Y_Train_dict] = prepareTrainDict(
        Y_Train, model_depth, signal_length, model_name)
    [Y_Test1, Y_Test_dict] = prepareTrainDict(
        Y_Test, model_depth, signal_length, model_name)
    #########################################################################################
    loss_weights = np.zeros(model_depth)
    for i in range(0, model_depth):
        loss_weights[i] = 1-(i*0.1)
    loss_weights
    #########################################################################################
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.001
    )

    if D_S == 0:
        # Build model for EEG Motion Artifact Removal - Deep Unet Architecture
        # AutoEncoder should be set at 'FALSE' for the Deep U-net model for Signal Reconstruction
        # D_S = 0, A_E = 0
        model = UNet(
            signal_length,
            model_depth,
            num_channel,
            model_width,
            kernel_size,
            problem_type=problem_type,
            output_nums=output_nums,
            ds=D_S,
            ae=A_E,
            ag=A_G,
            alpha=alpha
        ).UNet()

        model.compile(
            loss='mean_absolute_error',
            optimizer=optimizer,
            metrics=['mean_squared_error', 'accuracy']
        )
        #
    elif D_S == 1:
        # Build model for EEG Motion Artifact Removal - Deep Unet Architecture
        # AutoEncoder should be set at 'FALSE' for the Deep U-net model for Signal Reconstruction
        # D_S = 1, A_E = 0
        model = UNet(
            signal_length, model_depth,
            num_channel,
            model_width,
            kernel_size,
            problem_type=problem_type,
            output_nums=output_nums,
            ds=D_S,
            ae=A_E,
            ag=A_G,
            alpha=alpha
        ).MultiResUNet() # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

        model.compile(
            loss='mean_absolute_error',
            optimizer=optimizer,
            metrics=['mean_squared_error'],
            loss_weights=loss_weights
        )
    #################################################################################
    config = f'GRF_F{fold}_' +  model_name + str(signal_length) + '_' \
        + str(N_CHANNELS) + '_' + str(model_width) + '_' \
        + str(model_depth) + '_' + str(num_channel) + '_all_vel_' \
        + str(D_S) + str(A_G) + '_axis_corrected'
    #################################################################################
    if D_S == 0:
        callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, mode='min'), 
            tf.keras.callbacks.ModelCheckpoint(
                'trained_models/' + config +'.h5', 
                    verbose=1, 
                    monitor='val_loss', 
                    save_best_only=True, 
                    mode='min'
            )
        ]
        history = model.fit(
            X_Train, Y_Train, 
            epochs=EPOCHS, 
            batch_size=64, 
            verbose=1, 
            validation_split=0.2, 
            shuffle=True, 
            callbacks=callbacks
        )
        
    elif D_S == 1:
        callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_out_loss', patience=30, mode='min'), 
            tf.keras.callbacks.ModelCheckpoint(
                'trained_models/' + config + '.h5', 
                    verbose=1, 
                    monitor='val_loss', 
                    save_best_only=True, 
                    mode='min'
            )
        ]
        history = model.fit(
            X_Train1, 
            Y_Train_dict, 
            epochs=EPOCHS, 
            batch_size=64, 
            verbose=2, 
            validation_split=0.2, 
            shuffle=True, 
            callbacks=callbacks
        )
    #################################################################################
    if D_S == 0:
        GRF_pred = model.predict(X_Test, verbose=1)
        print(GRF_pred.shape)
    elif D_S == 1:
        GRF_pred = model.predict(X_Test1, verbose=1)
        print(GRF_pred[0].shape)
    #################################################################################
    ground_truth_mean = np.mean(Y_Test, axis=0)
    ground_truth_std = np.std(Y_Test, axis=0)
    prediction = np.nan_to_num(GRF_pred[0][:, :, 0])
    prediction_mean = np.mean(prediction, axis=0)
    prediction_std = np.std(prediction, axis=0)
    #################################################################################
    R = np.corrcoef(ground_truth_mean, prediction_mean)[0, 1]
    R2 = pearsonr(ground_truth_mean, prediction_mean)[0]
    #################################################################################
    plt.figure(figsize=(12, 7))
    x = np.linspace(0, 100, FRAME_LEN)
    plt.plot(x, ground_truth_mean, linewidth=3, label='Ground Truth')
    plt.fill_between(
        x, ground_truth_mean + ground_truth_std,
        ground_truth_mean - ground_truth_std, alpha=0.3,
        label='Ground Truth +/- STD'
    )
    plt.plot(x, prediction_mean, linewidth=3, label='Prediction')
    plt.fill_between(
        x, prediction_mean + prediction_std,
        prediction_mean - prediction_std, alpha=0.3,
        label='Prediction +/- STD'
    )
    # plt.legend()
    plt.title(f'R1 = {round(R, 5)}, R2 = {round(R2, 5)}')
    # plt.xlabel('Stance Phase (%)')
    # plt.ylabel('Moment Nm (Normalized)')
    plt.savefig(figures_dir + 'Results_' + config + '.png')
    plt.show()
    ################################################################################
    from shutil import copy
    copy('trained_models/' + config + '.h5', models_dir)
    joblib.dump(history, models_dir + \
                'History_' + config + '.joblib')
    ################################################################################
    history_plot(history)
    if D_S == 0:
        random_idx = random.sample(range(0, Y_Test.shape[0]), 6)
        plt.figure(figsize=(14, 9))
        for i in range(6):
            y_true = Y_Test[random_idx[i]]
            y_pred = GRF_pred[random_idx[i]]
            MAE = np.mean(np.abs(y_pred.ravel() - y_true.ravel()))
            plt.subplot(2, 3, i + 1)
            plt.plot(y_true, label='Ground Truth')
            plt.plot(y_pred.ravel(), label='Prediction')
            plt.title(f"MAE [{random_idx[i]}] : {MAE}")
            plt.legend();
        plt.show()
        
    elif D_S == 1:
        random_idx = random.sample(range(0, Y_Test.shape[0]), 6)
        plt.figure(figsize=(14, 9))
        for i in range(6):
            y_true = Y_Test[random_idx[i]]
            y_pred = GRF_pred[0][random_idx[i]]
            MAE = np.mean(np.abs(y_pred.ravel() - y_true.ravel()))
            plt.subplot(2, 3, i + 1)
            plt.plot(y_true, label='Ground Truth', linewidth=3)
            plt.plot(y_pred.ravel(), label='Prediction', linewidth=3)
            plt.title(f"Sample [{random_idx[i]}] MAE: {round(MAE, 4)}")
            plt.legend()
        plt.savefig(figures_dir + 'Examples_' + config + '.png')
        plt.show()

    #############################################################################
    construction_err, rmse, corr_coef, corr_coef2 = Construction_Error(Y_Test, GRF_pred[0])
    result = pd.DataFrame(np.array([construction_err, rmse, corr_coef]).T)
    results = pd.concat([results, result], axis=0)

    fold = fold + 1


==================================== FOLD 1 ====================================
(3136, 1024, 5)
(784, 1024, 5)
(3136, 1024)
(784, 1024)
Epoch 1/300

Epoch 00001: val_loss improved from inf to 1.10061, saving model to trained_models/GRF_F1_MultiResUNet1024_5_64_5_5_all_vel_10_axis_corrected.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


40/40 - 73s - loss: 1.8940 - out_loss: 0.4255 - level1_loss: 0.3639 - level2_loss: 0.3089 - level3_loss: 0.3106 - level4_loss: 0.3581 - level5_loss: 0.6005 - out_mean_squared_error: 0.5478 - level1_mean_squared_error: 0.5427 - level2_mean_squared_error: 0.4153 - level3_mean_squared_error: 0.5979 - level4_mean_squared_error: 1.1400 - level5_mean_squared_error: 1.4890 - val_loss: 1.1006 - val_out_loss: 0.2636 - val_level1_loss: 0.2202 - val_level2_loss: 0.2178 - val_level3_loss: 0.2449 - val_level4_loss: 0.2170 - val_level5_loss: 0.2122 - val_out_mean_squared_error: 0.0891 - val_level1_mean_squared_error: 0.0714 - val_level2_mean_squared_error: 0.0729 - val_level3_mean_squared_error: 0.0791 - val_level4_mean_squared_error: 0.0751 - val_level5_mean_squared_error: 0.0759 - 73s/epoch - 2s/step
Epoch 2/300

Epoch 00002: val_loss improved from 1.10061 to 1.09750, saving model to trained_models/GRF_F1_MultiResUNet1024_5_64_5_5_all_vel_10_axis_corrected.h5
40/40 - 30s - loss: 0.6789 - out_loss:

In [ ]:
joblib.dump(results, models_dir + 'Results_' + config + '.joblib')

In [ ]:
pd.DataFrame(np.array([construction_err, rmse, corr_coef]).T)

In [ ]:
while True:
    pass